In [64]:
from pathlib import Path
import os, shutil, re
from dotenv import load_dotenv

from langchain.text_splitter            import CharacterTextSplitter
from langchain.embeddings               import OpenAIEmbeddings
from langchain_community.vectorstores   import Chroma
from langchain.llms                     import OpenAI
from langchain.schema                   import Document
from langchain.memory                   import ConversationBufferMemory
from langchain.agents                   import Tool, AgentType, initialize_agent
from langchain.prompts                  import PromptTemplate
from langchain.chains                   import LLMChain
from langchain.retrievers               import BM25Retriever, EnsembleRetriever

import chromadb
from chromadb.config import Settings
from rapidfuzz import fuzz               


In [65]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

persist_dir = str(Path.home() / "chroma_asic_idx")
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)           # чистый билд


In [66]:
product_texts = [
    # ——— S19 Pro ———
    """
Bitmain Antminer S19 Pro 110 TH/s
Алгоритм SHA-256 (Bitcoin/BCH)
110 TH/s ±3 % • 3250 Вт ±5 % • 29,5 J/TH
Шум 75 дБ • 400×195×290 мм, 13,2 кг
Цена 199 000 ₽  (скидка от 3 шт)
Гарантия 12 мес. от Bitmain
Доставка СДЭК РФ / самовывоз (Москва)
    """,
    # ——— M30S++ ———
    """
MicroBT Whatsminer M30S++ 112 TH/s
SHA-256 • 112 TH/s ±2 % • 3472 Вт • 31 J/TH
Состояние БУ 2023, 1000 ч • гарантия 3 мес
Цена 128 000 ₽ • скидка 5 % ≥ 5 шт
Оплата BTC, Сбер, Tinkoff
Доставка Boxberry, ПЭК
    """,
    # ——— iPollo V1 Mini ———
    """
iPollo V1 Mini ETC 300 MH/s (Wi-Fi)
EtHash ETC • 300 MH/s ±10 % • 240 Вт
Шум 50 дБ • 178×143×90 мм, 2,1 кг
Цена 38 500 ₽ • гарантия 6 мес iPollo
Оплата USDT (TRC-20), карта РФ
Доставка EMS, Boxberry — в день оплаты
    """,
]
documents = [Document(page_content=txt.strip()) for txt in product_texts]


In [67]:
splitter = CharacterTextSplitter(
    separator="\n", chunk_size=800, chunk_overlap=150
)
chunks = splitter.split_documents(documents)
print("Чанков:", len(chunks))

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Chroma.from_documents(
    chunks,
    embedding        = embeddings,
    collection_name  = "asic_store",
    persist_directory= persist_dir,
    client_settings  = Settings(anonymized_telemetry=False),
)
print("Записано в Chroma:", vectorstore._collection.count())


Чанков: 3
Записано в Chroma: 15


In [68]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import BaseRetriever, Document
from rapidfuzz import fuzz
from pydantic import PrivateAttr

bm25 = BM25Retriever.from_documents(documents, k=3)

class FuzzyRetriever(BaseRetriever):
    """Простой retriever, ищущий по нечеткому совпадению substrings."""
    k: int = 3
    _docs: list[Document] = PrivateAttr(default_factory=list)

    def __init__(self, docs, k: int = 3, **kwargs):
        super().__init__(k=k, **kwargs)
        self._docs = docs

    def _get_relevant_documents(self, query: str, *, run_manager=None, **kwargs):
        scored = sorted(
            self._docs,
            key=lambda d: fuzz.partial_ratio(query.lower(), d.page_content.lower()),
            reverse=True,
        )
        return scored[: self.k]

    async def _aget_relevant_documents(self, query: str, *, run_manager=None, **kwargs):
        return self._get_relevant_documents(query)

fuzzy_retr = FuzzyRetriever(docs=documents, k=3)

vec_retr = vectorstore.as_retriever(search_kwargs={"k": 8})

hybrid_retriever = EnsembleRetriever(
    retrievers=[vec_retr, bm25, fuzzy_retr],
    weights=[0.5, 0.35, 0.15],
)


In [69]:
llm = OpenAI(temperature=0.0)


In [70]:
def product_info(question: str) -> str:
    q = re.sub(r"[^\w\s+]", " ", question.lower()).strip()
    docs = hybrid_retriever.get_relevant_documents(q)
    if not docs:
        return "Информация не найдена."

    context = "\n---\n".join(d.page_content for d in docs)
    prompt = (
        "Ты консультант по ASIC-майнерам.\n"
        "Отвечай ТОЛЬКО фактами из контекста ниже.\n"
        "Отвечай только на русском"
        "Если ответа нет — скажи: «Информация не найдена.»\n\n"
        f"Контекст:\n{context}\n\nВопрос: {question}\nОтвет:"
    )
    return llm.invoke(prompt).strip()

product_tool = Tool(
    name        = "product_info",
    func        = product_info,
    description = "Возвращает факты о майнерах (характеристики, цена, наличие)",
)


In [71]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(
    tools   = [product_tool],
    llm     = llm,
    agent   = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory  = memory,
    verbose = True,
)


In [72]:
print("🟢 Готов!  Пишите вопросы, 'exit' — выйти.")
while True:
    user = input("\nВы: ")
    if user.lower() in {"exit", "quit"}: break
    print("🤖:", agent.invoke({"input": user})["output"])


🟢 Готов!  Пишите вопросы, 'exit' — выйти.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: product_info
Action Input: models
Observation: iPollo V1 Mini ETC 300 MH/s (Wi-Fi), Bitmain Antminer S19 Pro 110 TH/s, MicroBT Whatsminer M30S++ 112 TH/s
Thought: Do I need to use a tool? No
AI: Я продаю различные модели асиков, включая iPollo V1 Mini ETC 300 MH/s (Wi-Fi), Bitmain Antminer S19 Pro 110 TH/s и MicroBT Whatsminer M30S++ 112 TH/s. Какая модель вас интересует?

> Finished chain.
🤖: Я продаю различные модели асиков, включая iPollo V1 Mini ETC 300 MH/s (Wi-Fi), Bitmain Antminer S19 Pro 110 TH/s и MicroBT Whatsminer M30S++ 112 TH/s. Какая модель вас интересует?


> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: product_info
Action Input: Bitmain Antminer S19 Pro
Observation: Название: Bitmain Antminer S19 Pro 110 TH/s
Ключевые слова: asic, асик, asics, miner, майнер, битмайн, bitmain, s19
Алгоритм: SHA-256 (Bitcoin,